In [29]:
import c3d
import numpy as np
import pandas as pd
import plotly.express as px

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display
import os
from read_mocap import PATH_TO_DATA, PATH_TO_RECORDED_DATA, read_and_preprocess_csv_data
pd.options.plotting.backend = "plotly"

In [23]:

print(f'path to recorded data = {PATH_TO_RECORDED_DATA}')
# Get the csv records only
records = [f for f in sorted(os.listdir(PATH_TO_RECORDED_DATA)) if f.endswith('.csv')]
print(f'recorded list = {records}')
all_data = read_and_preprocess_csv_data(os.path.join(PATH_TO_RECORDED_DATA,records[-1]))
pih_data = all_data.iloc[:, :9]
arm_data = all_data.iloc[:, 10:]
print(f' all data records:\t {all_data.columns}')
print(f'Rigid body records:\t {pih_data.columns}')
print(f'Human arm records: \t {arm_data.columns}')
display(arm_data)
wrst_vec = arm_data.loc[:,['arm:wrist_X_Position','arm:wrist_Y_Position', 'arm:wrist_Z_Position']].to_numpy()
print(wrst_vec)

path to recorded data = Data/human arm tracks
recorded list = ['Take 2022-12-14 05.13.01 PM_001.csv', 'Take 2023-01-18 07.26.05 PM_001.csv', 'Take 2023-01-18 3.csv', 'Take 2023-01-18 4.csv', 'Take 2023-01-18 5.csv', 'Take 2023-01-18 6.csv', 'Take 2023-01-18 7.csv', 'Take 2023-01-18 8.csv', 'Take 2023-01-19 1.csv', 'Take 2023-01-19 2 processed.csv', 'Take 2023-01-19 3 processed.csv', 'Take 2023-01-19 4 processed .csv']
 all data records:	 Index(['Frame', 'Time (Seconds)', 'PIH_X_Rotation', 'PIH_Y_Rotation',
       'PIH_Z_Rotation', 'PIH_W_Rotation', 'PIH_X_Position', 'PIH_Y_Position',
       'PIH_Z_Position', 'PIH_Error_per_Marker', 'arm:elbow_X_Position',
       'arm:elbow_Y_Position', 'arm:elbow_Z_Position',
       'arm:shoulder_X_Position', 'arm:shoulder_Y_Position',
       'arm:shoulder_Z_Position', 'arm:wrist_X_Position',
       'arm:wrist_Y_Position', 'arm:wrist_Z_Position'],
      dtype='object')
Rigid body records:	 Index(['Frame', 'Time (Seconds)', 'PIH_X_Rotation', 'PIH_Y_Rota

,arm:elbow_X_Position,arm:elbow_Y_Position,arm:elbow_Z_Position,arm:shoulder_X_Position,arm:shoulder_Y_Position,arm:shoulder_Z_Position,arm:wrist_X_Position,arm:wrist_Y_Position,arm:wrist_Z_Position
0,-5.857788,2.481445,-2.337769,-10.733032,-0.490967,-4.288208,-7.163635,-0.961548,-4.435669
1,-5.850952,2.481201,-2.337158,-10.721008,-0.487305,-4.285645,-7.155457,-0.958618,-4.432861
2,-5.844116,2.480957,-2.336548,-10.709045,-0.483887,-4.283203,-7.147278,-0.955688,-4.430176
3,-5.837280,2.480957,-2.336060,-10.697144,-0.480469,-4.280640,-7.139099,-0.952881,-4.427368
4,-5.830444,2.480957,-2.335449,-10.685181,-0.476807,-4.278076,-7.130920,-0.949829,-4.424561
...,...,...,...,...,...,...,...,...,...
7504,1.312256,5.511230,-21.920898,1.148958,2.868973,-0.218036,-10.510254,0.637573,-15.003174
7505,1.311768,5.514404,-21.942017,1.148958,2.868973,-0.218036,-10.523193,0.636230,-15.016968
7506,1.311279,5.517822,-21.963135,1.148958,2.868973,-0.218036,-10.536011,0.634888,-15.030884
7507,1.310547,5.521240,-21.984375,1.148958,2.868973,-0.218036,-10.548828,0.633545,-15.044678


[[ -7.163635  -0.961548  -4.435669]
 [ -7.155457  -0.958618  -4.432861]
 [ -7.147278  -0.955688  -4.430176]
 ...
 [-10.536011   0.634888 -15.030884]
 [-10.548828   0.633545 -15.044678]
 [-10.561768   0.632202 -15.058472]]


### Check if there are any NaNs left after interpolation 

In [28]:
all_data.isnull().sum()

Frame                      0
Time (Seconds)             0
PIH_X_Rotation             0
PIH_Y_Rotation             0
PIH_Z_Rotation             0
PIH_W_Rotation             0
PIH_X_Position             0
PIH_Y_Position             0
PIH_Z_Position             0
PIH_Error_per_Marker       0
arm:elbow_X_Position       0
arm:elbow_Y_Position       0
arm:elbow_Z_Position       0
arm:shoulder_X_Position    0
arm:shoulder_Y_Position    0
arm:shoulder_Z_Position    0
arm:wrist_X_Position       0
arm:wrist_Y_Position       0
arm:wrist_Z_Position       0
dtype: int64

# Calculate distance feature

In [35]:
def calc_dist_feature(df):
    '''
    This function takes the data frame as input where we have coordinates
    for target (pih_data[x,y,z]) and wrist (human arm end point) wirst[x,y,z]
    returns the distance between the human arm and the target tool.
    '''
    wrst_vec = df.loc[:,['arm:wrist_X_Position','arm:wrist_Y_Position', 'arm:wrist_Z_Position']].to_numpy()
    pih_vec = df.loc[:,['PIH_X_Position', 'PIH_Y_Position','PIH_Z_Position']].to_numpy()
    distance = np.array([])
    for wrst_pt, pih_pt in zip(wrst_vec, pih_vec):
        # if wrst_pt is not None and pih_pt is not None:
        dist =  np.linalg.norm(wrst_pt - pih_pt)
        distance = np.append(distance,dist)
    return distance
dist = calc_dist_feature(all_data)
print(min(dist))
all_data['Distance'] = dist
display(all_data)
print(all_data.columns)

0.3869343387128136


,Frame,Time (Seconds),PIH_X_Rotation,PIH_Y_Rotation,PIH_Z_Rotation,PIH_W_Rotation,PIH_X_Position,PIH_Y_Position,PIH_Z_Position,PIH_Error_per_Marker,arm:elbow_X_Position,arm:elbow_Y_Position,arm:elbow_Z_Position,arm:shoulder_X_Position,arm:shoulder_Y_Position,arm:shoulder_Z_Position,arm:wrist_X_Position,arm:wrist_Y_Position,arm:wrist_Z_Position,Distance
0,0,0.00,0.014209,-0.395462,0.022478,-0.918097,0.310257,1.581042,0.853389,0.001870,-5.857788,2.481445,-2.337769,-10.733032,-0.490967,-4.288208,-7.163635,-0.961548,-4.435669,9.502524
1,1,0.01,0.014209,-0.395462,0.022478,-0.918097,0.310257,1.581042,0.853389,0.001870,-5.850952,2.481201,-2.337158,-10.721008,-0.487305,-4.285645,-7.155457,-0.958618,-4.432861,9.493745
2,2,0.02,0.014209,-0.395462,0.022478,-0.918097,0.310257,1.581042,0.853389,0.001870,-5.844116,2.480957,-2.336548,-10.709045,-0.483887,-4.283203,-7.147278,-0.955688,-4.430176,9.485035
3,3,0.03,0.014209,-0.395462,0.022478,-0.918097,0.310257,1.581042,0.853389,0.001870,-5.837280,2.480957,-2.336060,-10.697144,-0.480469,-4.280640,-7.139099,-0.952881,-4.427368,9.476290
4,4,0.04,0.014209,-0.395462,0.022478,-0.918097,0.310257,1.581042,0.853389,0.001870,-5.830444,2.480957,-2.335449,-10.685181,-0.476807,-4.278076,-7.130920,-0.949829,-4.424561,9.467480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7504,7504,75.04,-0.010130,0.397558,-0.022317,0.917250,0.310537,1.581743,0.853764,0.000425,1.312256,5.511230,-21.920898,1.148958,2.868973,-0.218036,-10.510254,0.637573,-15.003174,19.220392
7505,7505,75.05,-0.010130,0.397558,-0.022317,0.917250,0.310537,1.581743,0.853764,0.000425,1.311768,5.514404,-21.942017,1.148958,2.868973,-0.218036,-10.523193,0.636230,-15.016968,19.239123
7506,7506,75.06,-0.010130,0.397558,-0.022317,0.917250,0.310537,1.581743,0.853764,0.000425,1.311279,5.517822,-21.963135,1.148958,2.868973,-0.218036,-10.536011,0.634888,-15.030884,19.257886
7507,7507,75.07,-0.010130,0.397558,-0.022317,0.917250,0.310537,1.581743,0.853764,0.000425,1.310547,5.521240,-21.984375,1.148958,2.868973,-0.218036,-10.548828,0.633545,-15.044678,19.276549


Index(['Frame', 'Time (Seconds)', 'PIH_X_Rotation', 'PIH_Y_Rotation',
       'PIH_Z_Rotation', 'PIH_W_Rotation', 'PIH_X_Position', 'PIH_Y_Position',
       'PIH_Z_Position', 'PIH_Error_per_Marker', 'arm:elbow_X_Position',
       'arm:elbow_Y_Position', 'arm:elbow_Z_Position',
       'arm:shoulder_X_Position', 'arm:shoulder_Y_Position',
       'arm:shoulder_Z_Position', 'arm:wrist_X_Position',
       'arm:wrist_Y_Position', 'arm:wrist_Z_Position', 'Distance'],
      dtype='object')


In [37]:
all_data.loc[:,['Distance']].plot()